In [ ]:
#####制作 train 和 val 的coco格式
import json
import os
import random

path = r"/home/admin/jupyter/Demo/DataSets/Data/images_small_small_json/"
save_path = r"/home/admin/jupyter/Demo/DataSets/Data/"


# 改 比例 ，路径（路径是 正负样本在一起的jaon文件夹），保存的名字

def make_coco(name_lisy,save_name,cls=[1,2,3,4,5,6]):

    coco = {}
    coco['info'] = {"year": 2019, "contributor": "AI3D", "date_created": "2019/10/21", "version": "2.0", "url": "", "description": ""}
    coco['categories'] = [{"id": 1, "name": "ASC-H", "supercategory": "abnormal"},{"id": 2, "name": "ASC-US", "supercategory": "abnormal"},
                         {"id": 3, "name": "HSIL", "supercategory": "abnormal"},{"id": 4, "name": "LSIL", "supercategory": "abnormal"},
                         {"id": 5, "name": "Candida", "supercategory": "normal"},{"id": 6, "name": "Trichomonas", "supercategory": "normal"}]
    coco['licenses'] =  [{"url": "", "id": 1, "name": ""}]
    assert isinstance(cls,list),'cls should be a list'
    #不需要修改coco['categories']
#     coco['categories']=[coco['categories'][i-1] for i in cls] ##wei_added
    print(coco['categories'])
#"ASC-H","ASC-US","HSIL","LSIL","Candida","Trichomonas",
    images = []
    annotations = []
    i = 1
    j = 1
    
    category = {"ASC-H":1,"ASC-US":2,"HSIL":3,"LSIL":4,"Candida":5,"Trichomonas":6}
    for name1 in name_lisy:
        with open(path + "/" + name1, "r") as f:
            js = json.load(f)
        if "kfb" in name1:
            name = name1.split('.')[0]+"."+name1.split('.')[1]  ###因为名字不太规范
        else:
            name = name1.split('.')[0]

        im_dic = dict()
        im_dic['file_name'] = name + '.jpg'
        im_dic['width'] = js['size'][0]
        im_dic['height'] = js['size'][1]
        im_dic['id'] = i  # 自动增加
        im_dic['license'] = 1

        images.append(im_dic)

        if js['has_box'] == False:
            i += 1
            continue

        for box in js['boxes']:
            an_dic = dict()
            if category[box['class']] not in cls: #wei_added
                continue
            an_dic['category_id'] = category[box['class']]
            an_dic['area'] = box['w'] * box['h']
            an_dic['segmentation'] = [
                [box['x'], box['y'], box['x'], box['y'] + box['h'], box['x'] + box['w'], box['y'] + box['h'],
                 box['x'] + box['w'], box['y'], ]]
            an_dic['iscrowd'] = 0
            an_dic['id'] = j  # 自动怎加
            an_dic['image_id'] = i  # 和上面对应
            an_dic['bbox'] = [box['x'], box['y'], box['w'], box['h']]

            annotations.append(an_dic)
            j += 1

        i += 1

    coco['images'] = images
    coco['annotations'] = annotations

    with open(save_path+save_name+'.json', "w") as f:
        json.dump(coco, f)
        
        
#category = {"ASC-H":1,"ASC-US":2,"HSIL":3,"LSIL":4,"Candida":5,"Trichomonas":6}            
#A  制作训练和验证集合
names = os.listdir(path)
random.shuffle(names)
print(len(names))
make_coco(names[:1000],'val_small_small')    #调节比例  现在1:6
make_coco(names[1000:],'train_small_small')



In [ ]:
######制作test 的 coco格式
import json
import os
import numpy as np


def make_coco(name_lisy,save_name):

    coco = {}
    coco['info'] = {"year": 2019, "contributor": "AI3D", "date_created": "2019/10/21", "version": "2.0", "url": "", "description": ""}
    coco['categories'] = [{"id": 1, "name": "ASC-H", "supercategory": "abnormal"},{"id": 2, "name": "ASC-US", "supercategory": "abnormal"},
                         {"id": 3, "name": "HSIL", "supercategory": "abnormal"},{"id": 4, "name": "LSIL", "supercategory": "abnormal"},
                         {"id": 5, "name": "Candida", "supercategory": "normal"},{"id": 6, "name": "Trichomonas", "supercategory": "normal"}]
    coco['licenses'] =  [{"url": "", "id": 1, "name": ""}]
    images = []
    annotations = []
    i = 1
    

    for name1 in name_lisy:
        with open(path  + name1, "r") as f:
            js = json.load(f)
        name = name1.split('.')[0]
        # print(name,i)
        im_dic = dict()
        an_dic = dict()
        im_dic['file_name'] = name + '.jpg'
        im_dic['width'] = js['size'][0]
        im_dic['height'] = js['size'][1]
        im_dic['id'] = i  # 自动增加
        im_dic['license'] = 1

        images.append(im_dic)
        
        # 假的注释

        an_dic['category_id'] = 1  
        an_dic['area'] = 100
        an_dic['segmentation'] = [0,0,0,10,10,10,10,0]  
        an_dic['iscrowd'] = 0
        an_dic['id'] = i  # 自动怎加
        an_dic['image_id'] = i  # 和上面对应
        an_dic['bbox'] = [0, 0, 10, 10]
        annotations.append(an_dic)

        i += 1
        

    coco['images'] = images
    coco['annotations'] = annotations

    with open(save_path+save_name + '.json', "w") as f:
        json.dump(coco, f)


#####注意路径！
path = r"/home/admin/jupyter/Demo/DataSets/Data/images_test_val_json/"

save_path = r"/home/admin/jupyter/Demo/DataSets/Data/"
names = os.listdir(path)

make_coco(names,'test_val')